In [21]:
import os
import numpy as np
import pandas as pd
from datetime import datetime


In [13]:
weather_folder_path = '../daily_weather_data/'
aqi_folder_path = '../hourly_aqi_data'

CITIES = [

    {"name": "Douala", "lat": 4.0483, "lon": 9.7043}, 
    {"name": "Yaoundé", "lat": 3.8667, "lon": 11.5167}, 
    {"name": "Bafoussam", "lat": 5.4737, "lon": 10.4179}, 
    {"name": "Bamenda", "lat": 5.9527, "lon": 10.1582}, 
    {"name": "Maroua", "lat": 10.591, "lon": 14.3159}, 
    {"name": "Ngaoundéré", "lat": 7.3167, "lon": 13.5833}, 
    {"name": "Buea", "lat": 4.1527, "lon": 9.241}, 
    {"name": "Ebolowa", "lat": 2.9, "lon": 11.15}, 
    {"name": "Garoua", "lat": 9.3, "lon": 13.4}, 
    {"name": "Bertoua", "lat": 4.5833, "lon": 13.6833}, 

]

In [14]:
def create_weather_df(folder_path, CITIES): 
    
    dataframes = []

    for city in CITIES: 
        city_name = city["name"]
        file_path = os.path.join(weather_folder_path, f"daily_dataframe_{city_name}.csv")

        df = pd.read_csv(file_path)

        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])

        df['city'] = city_name
        df['latitude'] = city['lat']
        df['longitude']= city['lon']

        dataframes.append(df)
    
    merged_df = pd.concat(dataframes, ignore_index= True)
    
    return merged_df


def create_aqi_df(folder_path, CITIES): 
    
    # Initialize an empty list to store DataFrames
    dataframes = []
    # Loop through all files in the folder
    for city in CITIES:
        # Construct the file path based on the city name
        city_name = city["name"]
        file_path = os.path.join(folder_path, f"hourly_aqi_{city_name}_dataframe.csv")

        # Read the CSV into a DataFrame
        df = pd.read_csv(file_path)

        #removing useless features...

        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])

        # Convert the 'date' column to datetime type
        df['date'] = pd.to_datetime(df['date'])

        # Group by date and calculate the mean of each feature for each day
        daily_df = df.groupby(df['date'].dt.date).mean()

        # Reset index without creating a new 'date' column
        daily_df = daily_df.reset_index(drop=True)

        # Add the date column back
        daily_df['date'] = df['date'].dt.date.unique()

        # Add city, latitude, and longitude columns
        daily_df['city'] = city_name
        daily_df['latitude'] = city['lat']
        daily_df['longitude'] = city['lon']

        # Append the DataFrame to the list
        dataframes.append(daily_df)

    # Merge all DataFrames into one big DataFrame
    merged_df = pd.concat(dataframes, ignore_index=True)
    # Ensure 'date' is the first column
    cols = ['date'] + [col for col in merged_df.columns if col != 'date']
    merged_df = merged_df[cols]
    merged_df =  merged_df.dropna(subset=['pm2_5', 'pm10'])

    
    return merged_df

weather_df = create_weather_df(folder_path = weather_folder_path, CITIES = CITIES)
aqi_df = create_aqi_df(folder_path = aqi_folder_path, CITIES = CITIES)

In [15]:
weather_df.shape, aqi_df.shape

((18270, 24), (5160, 14))

In [16]:
weather_df.head()

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,...,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,city,latitude,longitude
0,2018-12-31 23:00:00+00:00,2.0,31.203001,24.453001,27.290503,36.099533,29.665333,32.291813,0,0,...,0.0,0.0,16.610792,37.44,213.00797,17.15,3.764394,Douala,4.0483,9.7043
1,2019-01-01 23:00:00+00:00,55.0,29.553000,24.053000,26.646751,34.211563,29.438050,31.959915,0,0,...,0.0,8.0,16.099690,36.36,193.57042,15.36,3.223940,Douala,4.0483,9.7043
2,2019-01-02 23:00:00+00:00,51.0,30.403000,24.053000,26.725916,34.502907,29.304466,31.711157,0,0,...,0.0,5.0,16.418526,37.80,211.11131,14.54,3.167270,Douala,4.0483,9.7043
3,2019-01-03 23:00:00+00:00,51.0,30.303000,24.653000,27.050920,35.810380,30.148300,32.373516,0,0,...,0.0,1.0,15.946010,36.00,190.93540,16.67,3.538461,Douala,4.0483,9.7043
4,2019-01-04 23:00:00+00:00,51.0,31.053000,24.103000,27.178001,36.258404,28.911766,32.124626,0,0,...,0.0,1.0,20.268990,45.36,212.03770,17.73,3.800563,Douala,4.0483,9.7043


In [17]:
aqi_df.head()

,date,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,aerosol_optical_depth,dust,uv_index,uv_index_clear_sky,city,latitude,longitude
947,2022-08-04,19.475000,12.683333,192.458333,1.018750,0.229167,53.250000,0.281667,0.0,1.893750,2.347917,Douala,4.0483,9.7043
948,2022-08-05,21.087500,14.275000,213.916667,1.189583,0.210417,52.458333,0.259167,0.0,1.514583,2.300000,Douala,4.0483,9.7043
949,2022-08-06,14.512500,9.916667,201.500000,1.137500,0.210417,51.875000,0.145000,0.0,1.243750,2.377083,Douala,4.0483,9.7043
950,2022-08-07,20.895833,14.250000,211.583333,1.691667,0.470833,49.291667,0.156667,0.0,0.731250,2.456250,Douala,4.0483,9.7043
951,2022-08-08,21.708333,14.958333,231.250000,1.835417,0.575000,52.583333,0.212917,0.0,1.331250,2.427083,Douala,4.0483,9.7043


In [18]:
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date
weather_df.head()

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,...,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,city,latitude,longitude
0,2018-12-31,2.0,31.203001,24.453001,27.290503,36.099533,29.665333,32.291813,0,0,...,0.0,0.0,16.610792,37.44,213.00797,17.15,3.764394,Douala,4.0483,9.7043
1,2019-01-01,55.0,29.553000,24.053000,26.646751,34.211563,29.438050,31.959915,0,0,...,0.0,8.0,16.099690,36.36,193.57042,15.36,3.223940,Douala,4.0483,9.7043
2,2019-01-02,51.0,30.403000,24.053000,26.725916,34.502907,29.304466,31.711157,0,0,...,0.0,5.0,16.418526,37.80,211.11131,14.54,3.167270,Douala,4.0483,9.7043
3,2019-01-03,51.0,30.303000,24.653000,27.050920,35.810380,30.148300,32.373516,0,0,...,0.0,1.0,15.946010,36.00,190.93540,16.67,3.538461,Douala,4.0483,9.7043
4,2019-01-04,51.0,31.053000,24.103000,27.178001,36.258404,28.911766,32.124626,0,0,...,0.0,1.0,20.268990,45.36,212.03770,17.73,3.800563,Douala,4.0483,9.7043


In [23]:
comparison_date = datetime.strptime('2022-08-04', '%Y-%m-%d').date()

weather_df = weather_df[weather_df['date'] >= comparison_date]
weather_df.shape

(5150, 24)

In [36]:
full_data = pd.merge(
    weather_df, 
    aqi_df, 
    on=['date', 'city', 'latitude', 'longitude'], 
    how='inner'  # or 'outer', 'left', 'right' depending on your needs
)

In [37]:
full_data.shape

(5150, 34)

In [38]:
weather_df.shape

(5150, 24)

In [39]:
aqi_df.shape

(5160, 14)

In [40]:
full_data.head()

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,...,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,aerosol_optical_depth,dust,uv_index,uv_index_clear_sky
0,2022-08-04,51.0,27.753000,22.303,24.769667,31.690147,26.823467,28.843695,0,0,...,19.475000,12.683333,192.458333,1.018750,0.229167,53.250000,0.281667,0.0,1.893750,2.347917
1,2022-08-05,51.0,27.003000,22.903,24.638418,30.849144,27.140068,28.609718,0,0,...,21.087500,14.275000,213.916667,1.189583,0.210417,52.458333,0.259167,0.0,1.514583,2.300000
2,2022-08-06,55.0,28.203001,22.153,24.178001,33.463192,26.015505,28.399092,0,0,...,14.512500,9.916667,201.500000,1.137500,0.210417,51.875000,0.145000,0.0,1.243750,2.377083
3,2022-08-07,51.0,27.653000,22.603,24.663420,32.916840,26.624214,28.979774,0,0,...,20.895833,14.250000,211.583333,1.691667,0.470833,49.291667,0.156667,0.0,0.731250,2.456250
4,2022-08-08,51.0,27.703001,23.003,24.994665,31.993069,27.470306,29.261251,0,0,...,21.708333,14.958333,231.250000,1.835417,0.575000,52.583333,0.212917,0.0,1.331250,2.427083


In [41]:
full_data.to_csv('full_data.csv')